### Testing custom package

In [ ]:
from nav_pii_anon.spacy.spacy_model import SpacyModel
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
tx = ("Mitt navn er Marius og mitt personnummer er 15044216652. "
      "Telefonnummeret mitt er 94867658 og jeg reiser "
      "hjem 24.07.2020")

In [ ]:
nlp = SpacyModel()

In [ ]:
graph = nlp.dependency_graph(tx)

In [ ]:
nx.draw_shell(graph, node_size=15)

In [ ]:
nx.draw(graph, with_labels=True)

In [ ]:
nlp.top_n_nodes(tx, 4)

In [ ]:
sorted(list(graph.degree), key= lambda x: x[1], reverse=True)

In [ ]:
nlp.similarity(tx)

In [ ]:
nlp.predict(tx)

In [ ]:
nlp.add_patterns()

In [ ]:
nlp.replace(tx)

In [ ]:
nlp.predict(tx)

In [ ]:
nlp.display_predictions(tx)